<a href="https://colab.research.google.com/github/Harooniqbal4879/AgenticAI/blob/main/Agentic_RAG_Starter_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 Agentic RAG Assistant (LangChain + OpenAI + FAISS)

This Colab notebook demonstrates a starter project for an **Agentic AI-powered Retrieval-Augmented Generation (RAG)** assistant using:
- LangChain
- OpenAI GPT-3.5/4
- FAISS (Vector Store)
- PDF Document Loader & Text Chunker

*Generated on June 18, 2025*


In [ ]:

# 📦 Install Required Packages
!pip install -U langchain-community openai faiss-cpu tiktoken pypdf requests


In [2]:

# 🔑 Set Your OpenAI API Key
import os
os.environ['OPENAI_API_KEY'] = "sk-..."  # Replace with your key


In [6]:

# 📄 Load PDF and Chunk Text
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

pdf_path = "/content/sample_data/Testing_RAG.pdf"  # Upload your PDF via sidebar
loader = PyPDFLoader(pdf_path)
pages = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(pages)
print(f"📚 Loaded and chunked {len(docs)} documents")


📚 Loaded and chunked 0 documents


In [ ]:

# 🔗 Embed and Store in FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embeddings)


In [ ]:

# 🧠 Refine User Query with LLM
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.3)
refine_prompt = PromptTemplate(
    input_variables=["question"],
    template="Refine the user's question for clarity and specificity:\n\nUser: {question}\n\nRefined:"
)
refine_chain = LLMChain(llm=llm, prompt=refine_prompt)


In [ ]:

# 🔍 Build the RetrievalQA Chain (RAG)
from langchain.chains import RetrievalQA

retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)


In [ ]:

# ❓ Ask a Question
user_question = "What are the biggest staffing challenges in 2024?"
refined_q = refine_chain.run(user_question)

print("🔁 Refined Question:", refined_q)
response = qa_chain.run(refined_q)
print("\n💡 Answer:\n", response)
